In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
import seaborn as sb

In [52]:
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')


In [53]:
data = "data/train.csv"
blackfriday = pd.read_csv(data)
blackfriday.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [54]:
def setNumericValues(dataFrame: pd.DataFrame, column: str, options: "list[str]"):
    index = 0
    for option in options:
        index += 1
        dataFrame.loc[dataFrame[column] == option, column] = index

genders = ['M', 'F']
ages = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
cityCategory = ['A', 'B', 'C']
stayYears = ['0', '1', '2', '3', '4+']

setNumericValues(blackfriday, 'Gender', genders)
setNumericValues(blackfriday, 'Age', ages)
setNumericValues(blackfriday, 'City_Category', cityCategory)
setNumericValues(blackfriday, 'Stay_In_Current_City_Years', stayYears)

# blackfriday.loc[blackfriday['Gender'] == 'M', 'Gender'] = 1
# blackfriday.loc[blackfriday['Gender'] == 'F', 'Gender'] = 2

# blackfriday.loc[blackfriday['Age'] == '0-17', 'Age'] = 1
# blackfriday.loc[blackfriday['Age'] == '18-25', 'Age'] = 2
# blackfriday.loc[blackfriday['Age'] == '26-35', 'Age'] = 3
# blackfriday.loc[blackfriday['Age'] == '36-45', 'Age'] = 4
# blackfriday.loc[blackfriday['Age'] == '46-50', 'Age'] = 5
# blackfriday.loc[blackfriday['Age'] == '51-55', 'Age'] = 6
# blackfriday.loc[blackfriday['Age'] == '55+', 'Age'] = 7

newDataFrame = blackfriday.drop(columns=['Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Product_ID'])
newDataFrame = newDataFrame.groupby(['User_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status']).sum('Purchase').reset_index()
newDataFrame.drop(columns=['User_ID'], inplace=True)

newDataFrame.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase
0,2,1,10,1,3,0,334093
1,1,7,16,3,5,0,810472
2,1,3,15,1,4,0,341635
3,1,5,7,2,3,1,206468
4,1,3,20,1,2,1,821001


In [55]:
blackfriday['Stay_In_Current_City_Years'].value_counts()

2    193821
3    101838
4     95285
5     84726
1     74398
Name: Stay_In_Current_City_Years, dtype: int64

In [57]:
bf_features = newDataFrame[['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status']].values
bf_target = newDataFrame[['Purchase']].values
X, y = scale(bf_features), bf_target

LinearPrediction = LinearRegression()
LinearPrediction.fit(X, y)
print(LinearPrediction.score(X, y))

0.13098441541933248
